# Agent Handoffs - Learning Objectives

This notebook demonstrates how to implement agent handoffs in AI applications using the OpenAI Agents SDK. By the end of this notebook, you'll understand:

1. **Agent Handoff Architecture** - How to design and implement a system where multiple specialized agents collaborate on complex tasks
   
2. **Agent Specialization** - How to create agents with specific capabilities and responsibilities:
   - Content generation (Dad Jokes Agent)
   - Response formatting (User Response Writer)
   - Task orchestration (Orchestrator Agent)

3. **Workflow Orchestration** - How to design an agent that can delegate tasks to specialized agents based on user requests

4. **Structured Output Handling** - How to define and use Pydantic models for structured agent responses

5. **Agent Communication Patterns** - How to implement bidirectional handoffs between agents to create complete workflows

6. **Workflow Visualization** - How to create visual representations of agent communication pathways

7. **Interactive Conversation Management** - How to maintain conversation context through multiple agent handoffs


This notebook guides you through designing, implementing, and testing a multi-agent system with handoffs.

code references: 
https://github.com/openai/openai-agents-python/blob/main/examples/basic/agent_lifecycle_example.py 



In [1]:
from dotenv import load_dotenv
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    handoff,
    ModelSettings
)

from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents import handoff, RunContextWrapper


from pydantic import BaseModel, Field
load_dotenv()

True

In [2]:
from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace
from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph

In [3]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)

# Define the Agents


## Author Agent 

Dad Jokes agent generates new content. It procduces a structured output as instance of class `Joke`

Notice the instrutions and handoff description prompts. 

In [4]:


class Joke(BaseModel):
    joke: str = Field(description="The joke text")
    punchline: str = Field(description="The punchline of the joke")
    language: str = Field(description="The language of the joke")

dad_jokes_agent = Agent(
    name="data_jokes_author",
    instructions=(
        # f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an agent that specialises in creating jokes. "
        "You do not answer any other questions. "
        "always deliver a punchline. "
        " ## IMPORTANT ##"
        "Always respond in English, even if the user asks in a different language. "
        "you generate structured output that is not suitable for end user consumption. "
    ),
    handoff_description=(
        "An agent capable of creating new jokes based on user input "
    ),
    model=model_name,
    output_type=Joke,
)


## Response Writer agent

The response writer is responsible for producing a final response to the user. It takes the output of full execution context and turn it into a user-friendly message. 


In this case, behinging handed the full context is important. Handoffs will hand over the control with the full message history, which the writer in this case will need in order to produce the final response. 


The dad jokes generator agent didnot necessarly need to be aware of the full context, it could hav been implemented using the as_tool pattern as all it needed is the topic and language of the content to generate. 

In [5]:

user_response_agent = Agent[Joke](
    name="user_response_writer",
    instructions=(
        # f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an agent that writes messaages to be presented to end users."
        "your response always include summary of steps taken to produce the final output  "
        "in bullet points format."
        "Output must be formatted as markdown."
    ),
    handoff_description="An agent capable of generating final user responses from any input provided in a presentable format.",
    output_type=str,
    model=model_name,
)



## Orchestrator Agent


The orchestrator in this case is always handing off to one of the two agents to either generate content to produce a formatted user response.

The dad jokes agent handoffs was also updated to handdoff control back to the orchestrator. 

In [6]:

orchestrator_agent = Agent[Joke](
    name="orchestrator_agent",
    model=model_name,
    tool_use_behavior="run_llm_again",
    instructions=(
        # f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an orchestrator agent. "
        "You think about the user request, create a plan to solve the task, "
        "You have the options to pass the task to content generation agents, or " \
        "to user response writers, each specialise in their domain. "
        "You identiy the next agent to carry out the task then handoff to them"
        "You are not allowed to end the conversation directly, "
        "only user response writers are allowed to complete the conversation."
    ),
    handoff_description=(
        "An agent that orchestrates the task by defining an "
        "execution plan based on available agents and tools and inputs,"
        "then  handing off to other agents. "
        ),
    handoffs=[dad_jokes_agent,user_response_agent],
)


In [7]:
dad_jokes_agent.handoffs = [handoff(
    orchestrator_agent,
    tool_description_override="""
        An agent capable of routing the output to the correct
        next agent for packaing the response for delivery
        """
        )]

## Visualising the the Workflow

In [8]:
try:
    gv = draw_graph(
        orchestrator_agent,
        filename="./viz/07_agent_handoffs_orchestrator_agent_v2.gv",

    )

except Exception as e:
    print(f"Error: {e}")



Error: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


# Implementing Agent Handoffs with the Agents SDK

![handoff flow](./viz/07_agent_handoffsv2.svg)



In [9]:
from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

msg = input ("Enter your message: ") or "Hello, tell me a joke about Chickens and Quantum Computing"

inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

result = None
try:
    with trace(workflow_name="handoffs", group_id="mdsi") as tr:
        print(f"Trace URL: {get_trace_url(tr)}")
        result = await Runner.run(
            starting_agent=orchestrator_agent,
            input=inputs,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
                model_settings=ModelSettings(
                    **{"temperature": 0.2, "max_tokens": 5000}
                )),
            max_turns=5,
        )

        print(result.final_output)
        print(result.last_agent)
except Exception as e:
    print(f"Error: {e}")



Trace URL: https://platform.openai.com/traces/trace?trace_id=trace_88c1cdbae56c482396a68de3998a65a1
Why did the chicken get into quantum computing?

Because it wanted to be in a superposition of both crossing the road and not crossing the road at the same time!

---

### Steps Taken:
1. **Identified the Joke Theme**: Combined chickens and quantum computing.
2. **Incorporated Quantum Concept**: Used the idea of "superposition" from quantum mechanics.
3. **Added Humor**: Played on the classic "why did the chicken cross the road?" joke.
4. **Finalized the Joke**: Delivered a lighthearted punchline.
Agent(name='user_response_writer', instructions='You are an agent that writes messaages to be presented to end users.your response always include summary of steps taken to produce the final output  in bullet points format.Output must be formatted as markdown.', handoff_description='An agent capable of generating final user responses from any input provided in a presentable format.', handoffs=[]

## User Interaction 

Run the agent in a streemed output mode and interact with user to carry out a conversation.

In [ ]:
from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace
import uuid

conversation_id = str(uuid.uuid4().hex[:16])

msg = input("Hi! We speak French, Spanish and English. How can I help? ")
agent = orchestrator_agent
inputs: list[TResponseInputItem] = []
inputs.append({"content": msg, "role": "user"})

while True:
    # Each conversation turn is a single trace. Normally, each input from the user would be an
    # API request to your app, and you can wrap the request in a trace()




    with trace("Routing example", group_id=conversation_id) as tr:
        print(f"Trace URL: {get_trace_url(tr)}")
        result = Runner.run_streamed(
            agent,
            input=inputs,
            run_config=RunConfig(model=model_name, model_provider=GITHUB_MODEL_PROVIDER),
        )
        async for event in result.stream_events():
            if not isinstance(event, RawResponsesStreamEvent):
                continue
            data = event.data
            if isinstance(data, ResponseTextDeltaEvent):
                print(data.delta, end="", flush=True)
            elif isinstance(data, ResponseContentPartDoneEvent):
                print("\n")

    inputs = result.to_input_list()
    print("\n")


    agent = result.current_agent
    user_msg = input("Enter a message: ")
    if user_msg in ("exit", "quit", ""):
        break
    inputs.append({"content": user_msg, "role": "user"})